In [3]:
## - REQUIRED LIBRARIES
import numpy as np
from scipy import stats

# Mean-Variance Analysis and Performance Indicators

Consider a financial market with three risky assets whose characteristics are given below. Assume there exists a risk-free asset whose return is 3.5\%. Suppose you are an investor who seeks mean-variance optimal portfolios for a target expected return of 6.5\%.

In [4]:
## - INPUTS

# Mean return of the risky assets
mean_ret = np.array([0.07, 0.05, 0.08])

# Correlation matrix of the returns of the risky assets 
corr_ret = np.array([[1.00, 0.50, 0],
                    [0.50, 1.00, -0.75],
                    [0, -0.75, 1.00]])

# Standard deviations of the returns of the risky assets
sd_ret = np.array([1.5, 2.0, 1])

# Risk-free rate
r0 = 0.035

# Target expected return
target_mean = 0.065

#### Q1. Determine the composition of the minimum-variance portfolio with no risk-free asset and the standard deviation of its returns.

In [5]:
# Covariance matrix of the returns of the risky assets
cov_ret = np.diag(sd_ret) @ corr_ret @ np.diag(sd_ret)

In [6]:
# Inverse of the covariance matrix of returns
inv_cov_ret = np.linalg.inv(cov_ret)

In [7]:
# # Verify that the covariance matrix is positive definite
# np.all(np.linalg.eigvals(cov_ret) > 0)

In [8]:
# Define recurring quantities in the mean-variance analysis
vec1 = np.linspace(1, 1, len(mean_ret))
a = vec1.T @ inv_cov_ret @ vec1
b = vec1.T @ inv_cov_ret @ mean_ret

In [9]:
# Composition of the minimum-variance portfolio
phi_minvar = (1 / a) * inv_cov_ret @ vec1
sd_minvar = 1 / np.sqrt(a)

In [10]:
# Confirm that the risk-free rate satisfies the assumption
r0 < b/a

True

In [11]:
print(b/a)

0.06962499999999999


In [12]:
# FINAL ANSWER
print('Comaposition of min-var portfolio: ', phi_minvar)
print('Std dev of return of min-var portfolio: ', sd_minvar)

Comaposition of min-var portfolio:  [-0.2     0.4125  0.7875]
Std dev of return of min-var portfolio:  0.41079191812887456


#### Q2. Determine the composition of the mean-variance optimal portfolio with no risk-free asset for the target expected return of 6.5\%. Verify that the standard deviation of its return is approximately 57.4620\%.

In [13]:
# Composition of the self-financing portfolio
mean_omega = np.sqrt((mean_ret - (b / a) * vec1).T @ inv_cov_ret @ (mean_ret - (b / a) * vec1))
omega = inv_cov_ret @ (mean_ret - (b / a) * vec1) / mean_omega

In [14]:
# Composition of the MV-optimal portfolio with no risk-free asset
target_mean = 0.065
phi_mv = phi_minvar + ((target_mean - (b / a)) / mean_omega) * omega
sd_mv = np.sqrt(phi_mv.T @ cov_ret @ phi_mv) 

In [15]:
# FINAL ANSWER
print('Composition of MV-optimal NRF portfolio: ', phi_mv)
print('Std dev of return of MV-optimal NRF portfolio: ', sd_mv)

Composition of MV-optimal NRF portfolio:  [-0.33962264  0.61320755  0.72641509]
Std dev of return of MV-optimal NRF portfolio:  0.5746204653902276


#### Q3. Determine the risky allocation $\phi$ of the mean-variance optimal portfolio $\varphi$ with a risk-free asset for the target expected return of 6.5\%. Verify that the risk-free allocation of $\varphi$ is approximately 14.9437\% of the initial wealth.

In [16]:
# Composition of the tangent portfolio (risk allocation)
phi_tangent = (1 / (b - r0 * a)) * inv_cov_ret @ (mean_ret - r0 * vec1)
mean_tangent = r0 + (1 / (b - r0 * a)) * (mean_ret - r0 * vec1).T @ inv_cov_ret @ (mean_ret - r0 * vec1)
sd_tangent = (1 / (b - r0 * a)) * np.sqrt((mean_ret - r0 * vec1).T @ inv_cov_ret @ (mean_ret - r0 * vec1))

In [17]:
# Risky allocation of the MV-optimal portfolio with a risk-free asset
phi_mvrf = ((target_mean - r0) / (mean_tangent - r0)) * phi_tangent
sd_mvrf = np.sqrt(phi_mvrf.T @ cov_ret @ phi_mvrf)

phi_mvrf, 1 - sum(phi_mvrf)

(array([-0.15353122,  0.32702149,  0.67707267]), 0.1494370522006141)

In [18]:
phi_mvrf

array([-0.15353122,  0.32702149,  0.67707267])

In [19]:
# FINAL ANSWER
print('Risky allocation of MV-optimal WRF portfolio: ', phi_mvrf)
print('Risk-free allocation of MV-optimal WRF portfolio', 1 - sum(phi_mvrf))

Risky allocation of MV-optimal WRF portfolio:  [-0.15353122  0.32702149  0.67707267]
Risk-free allocation of MV-optimal WRF portfolio 0.1494370522006141


#### Q4. What is the maximum diversification ratio achievable within this financial market? Between the mean-variance optimal portfolios with and without a risk-free allocation, show that the portfolio without a risk-free allocation is less diversified.

In [20]:
# Maximum diversification ratio
dr_max = np.sqrt(sd_ret.T @ inv_cov_ret @ sd_ret)

# FINAL ANSWER
print('Maximum diversification ratio: ', dr_max)

Maximum diversification ratio:  3.214550253664318


In [21]:
# Diversification ratio of the portfolios above
dr_tangent = phi_tangent.T @ sd_ret / sd_tangent
dr_mv = phi_mv.T @ sd_ret / sd_mv
dr_mvrf = phi_mvrf.T @ sd_ret / sd_mvrf
dr_minvar = phi_minvar.T @ sd_ret / sd_minvar

#dr_tangent, dr_mv, dr_mvrf, dr_minvar

In [22]:
# FINAL ANSWER
print('DR of MV-optimal NRF portfolio: ', dr_mv)
print('DR of MV-optimal WRF portfolio: ', dr_mvrf)

DR of MV-optimal NRF portfolio:  2.511912320134428
DR of MV-optimal WRF portfolio:  3.121583769000124


In [23]:
# # Betas of the mean-variance optimal portfolios with respect to the benchmark
# beta_mv = phi_mv.T @ betas
# beta_mvrf = phi_mvrf.T @ betas
# beta_minvar = phi_minvar.T @ betas

# beta_mv, beta_mvrf, beta_minvar

#### Q5. Determine the beta of the MV-optimal portfolios with and without a risk-free asset and the minimum-variance portfolio with respect to the tangent portfolio. Verify that the Jensen indices of these three portfolios are equal to 0 (within numerical tolerance) and their Treynor indices are the same (within numerical error).

In [24]:
# Betas of the mean-variance optimal portfolios with respect to the tangent portfolio
beta_mv = phi_mv.T @ cov_ret @ phi_tangent / (sd_tangent ** 2)
beta_mvrf = phi_mvrf.T @ cov_ret @ phi_tangent / (sd_tangent ** 2)
beta_minvar = phi_minvar.T @ cov_ret @ phi_tangent / (sd_tangent ** 2)

# FINAL ANSWER - BETAS
beta_mv, beta_mvrf, beta_minvar

(0.8505629477993842, 0.8505629477993861, 0.9816914022517912)

In [25]:
# Jensen indices w.r.t. tangent portfolio
ji_mv = target_mean - (r0 + beta_mv * (mean_tangent - r0))
ji_mvrf = target_mean - (r0 + beta_mvrf * (mean_tangent - r0))
ji_minvar = (b / a) - (r0 + beta_minvar * (mean_tangent - r0))

# FINAL ANSWER - JENSEN INDEX
ji_mv, ji_mvrf, ji_minvar

(5.551115123125783e-17, -1.3877787807814457e-17, -2.7755575615628914e-17)

In [26]:
# Treynor indices w.r.t. tangent portfolio
ti_mv = (target_mean - r0) / beta_mv
ti_mvrf = (target_mean - r0) / beta_mvrf
ti_minvar = (b / a - r0) / beta_minvar

# FINAL ANSWER - TREYNOR INDEX
ti_mv, ti_mvrf, ti_minvar

(0.03527075812274375, 0.03527075812274367, 0.03527075812274367)

# Markowitz Risk Measures

#### Q6. Let $L(\phi) = - \phi^\top (R - r_0 \mathbf{1})$ denote the loss function associated to a portfolio with risky allocation $\phi \in \mathbb{R}^3$. Let $\alpha \in (\frac{1}{2},1)$ be a constant. Show that the $100\alpha\%$ value-at-risk (VaR) and expected shortfall (ES) associated to $L(\phi)$ are positive definite Markowitz risk measures.

**Solution.** Since $R \sim N(M, \Sigma)$, then $\mathsf{E}[L(\phi)] = -\phi^\top (M - r_0 \mathbf{1})$ and $\mathsf{Var}(L(\phi)) = \phi^\top \Sigma \phi$. Therefore,

$$\mathsf{VaR}_\alpha(L(\phi)) = -\phi^\top (M - r_0 \mathbf{1}) + F_Z^{-1}(\alpha) \sqrt{\phi^\top \Sigma \phi}$$

and

$$\mathsf{ES}_\alpha(L(\phi)) = -\phi^\top (M - r_0 \mathbf{1}) + \frac{f_Z(F_Z^{-1}(\alpha))}{1 - \alpha} \sqrt{\phi^\top \Sigma \phi},$$

where $f_Z$ and $F_Z$ denote the pdf and cdf, respectively, of $Z \sim N(0,1)$. Since $\alpha \in (\frac{1}{2},1)$, we have $F_Z^{-1}(\alpha) > 0$ and $\frac{f_Z(F_Z^{-1}(\alpha))}{1 - \alpha} > 0$. The condition $\lambda > 0 \|M - r_0 \mathbf{1}\|_{\Sigma^{-1}}$ can then be checked to verify that the two risk measures are positive definite Markowitz risk measures.

#### Q7. Consider the portfolio $\phi = (-0.2, 0.3, 0.9)^\top$ with no risk-free allocation. What is the 95\% ES associated to this portfolio?

In [29]:
# Inputs
portfolio = np.array([-0.2, 0.3, 0.9])
conf_level = 0.95

# Calculate ES
portfolio_ES = - portfolio.T @ (mean_ret - r0 * vec1) + stats.norm.pdf(stats.norm.ppf(conf_level)) / (1 - conf_level) * np.sqrt(portfolio.T @ cov_ret @ portfolio)

print('Portfolio 95% ES:')
print(portfolio_ES)

Portfolio 95% ES:
1.033817015207772


#### Q8. Use the Euler capital allocation rule to determine how much risk capital (as determined by the 95\% ES) must be allocated to each risky asset.

**Solution.** *(Verify this using Proposition 4.12 in the slides)* For any Markowitz risk measure $\mathcal{M}_\lambda(\phi)$, its gradient with respect to the portfolio risky allocation $\phi$ is given by

$$\nabla_\phi \mathcal{M}_\lambda(\phi) = - (M - r_0 \mathbf{1}) + \lambda \frac{\Sigma \phi}{\sqrt{\phi^\top \Sigma \phi}}.$$

The $i$th component of the gradient is given by

$$\frac{\partial \mathcal{M}_\lambda(\phi)}{\partial \phi_i} = \mathbf{e}_i^\top (\nabla_\phi \mathcal{M}_\lambda(\phi)) = -(M_i - r_0) + \lambda \frac{\mathbf{e}_i^\top \Sigma \phi}{\sqrt{\phi^\top \Sigma \phi}}$$

where $\mathbf{e}_i$ is the $i$th elementary basis vector of $\mathbb{R}^d$ (i.e. $1$ in the $i$th position, 0 elsewhere). The Euler risk allocation to the $i$th asset is therefore given by $\phi_i \frac{\partial \mathcal{M}_\lambda(\phi)}{\partial \phi_i}$, where the $\lambda$ in the Markowitz risk measure for $\mathsf{ES}_\alpha(L(\phi))$ is used. 

In [34]:
# ES gradient
portfolio_ret_var = portfolio.T @ cov_ret @ portfolio
ES_gradient = -(mean_ret - r0 * vec1) + stats.norm.pdf(stats.norm.ppf(conf_level, 0, 1), 0, 1) / (1 - conf_level) * cov_ret @ portfolio / np.sqrt(portfolio_ret_var)

# Euler risk allocations
ES_Euler_allocation = ES_gradient * portfolio

print('Euler allocations of 95% ES:')
print(ES_Euler_allocation)

Euler allocations of 95% ES:
[ 0.007      -0.54040851  1.56722552]
